In [2]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding='latin-1')
print(df.columns)


Index(['text', 'target'], dtype='object')


In [4]:
from transformers import pipeline
from sklearn.metrics import classification_report

# 2. Charger un modèle LLM (ex. FLAN-T5)
classifier = pipeline("text2text-generation", model="google/flan-t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
# 3. Fonction de classification avec prompt engineering
def classify_email(text):
    prompt = f"Classify the following email as 'spam' or 'ham': {text}"
    result = classifier(prompt, max_length=10)[0]['generated_text']
    return result.strip().lower()

In [6]:
# 4. Appliquer le modèle sur quelques exemples (attention : long si dataset complet)
df_sample = df.sample(100, random_state=42).copy()  # sous-échantillonner pour éviter lenteur
df_sample['predicted'] = df_sample['text'].apply(classify_email)


Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

In [7]:
# 5. Évaluer les performances
print(classification_report(df_sample['target'].str.lower(), df_sample['predicted'], zero_division=0))
print(df_sample[['text', 'target', 'predicted']].head(10))

              precision    recall  f1-score   support

         ham       1.00      0.12      0.22        88
        spam       0.13      1.00      0.24        12

    accuracy                           0.23       100
   macro avg       0.57      0.56      0.23       100
weighted avg       0.90      0.23      0.22       100

                                                   text target predicted
3245  Funny fact Nobody teaches volcanoes 2 erupt, t...    ham      spam
944   I sent my scores to sophas and i had to do sec...    ham      spam
1044  We know someone who you know that fancies you....   spam      spam
2484  Only if you promise your getting out as SOON a...    ham      spam
812   Congratulations ur awarded either Ã¥Â£500 of C...   spam      spam
2973         I'll text carlos and let you know, hang on    ham      spam
2991          K.i did't see you.:)k:)where are you now?    ham      spam
2942             No message..no responce..what happend?    ham       ham
230   Get down i

In [9]:
pip install streamlit transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [11]:
!pip install streamlit transformers pyngrok


In [12]:
code = '''
import streamlit as st
from transformers import pipeline

st.title("🤖 Chatbot LLM - Hugging Face")

@st.cache_resource
def load_pipeline():
    return pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=100)

llm_pipeline = load_pipeline()

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.text_input("You:", "")

if user_input:
    st.session_state.chat_history.append(("🧑", user_input))
    prompt = f"You are a helpful assistant. Answer the question: {user_input}"
    response = llm_pipeline(prompt)[0]['generated_text']
    st.session_state.chat_history.append(("🤖", response))

for speaker, message in st.session_state.chat_history:
    st.markdown(f"**{speaker}**: {message}")
'''

with open("chatbot_app.py", "w") as f:
    f.write(code)


In [13]:
from pyngrok import ngrok
import threading
import os

# Ouvrir un tunnel sur le port 8501
public_url = ngrok.connect(port=8501)
print("🔗 URL publique :", public_url)

# Lancer Streamlit dans un thread séparé
def run():
    os.system("streamlit run chatbot_app.py")

thread = threading.Thread(target=run)
thread.start()


ERROR:pyngrok.process.ngrok:t=2025-07-08T13:18:17+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-08T13:18:17+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-08T13:18:17+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.